In [5]:
from torchtext import data
from torchtext import datasets
import spacy

In [27]:
import torch.nn as nn

In [34]:
a = nn.Embedding(20, 300)

In [37]:
a.num_embeddings

20

In [8]:
!python -m spacy download en
!python -m spacy download de

    100% |████████████████████████████████| 37.4MB 56.3MB/s ta 0:00:011    82% |██████████████████████████▎     | 30.7MB 36.2MB/s eta 0:00:01

    Linking successful
    /Users/sob/Desktop/cs287/homeworks/env/lib/python3.6/site-packages/en_core_web_sm
    -->
    /Users/sob/Desktop/cs287/homeworks/env/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')

    100% |████████████████████████████████| 38.2MB 47.1MB/s ta 0:00:011 1% |▍                               | 440kB 7.3MB/s eta 0:00:06    96% |███████████████████████████████ | 37.1MB 49.8MB/s eta 0:00:01
  Running setup.py install for de-core-news-sm ... done

    Linking successful
    /Users/sob/Desktop/cs287/homeworks/env/lib/python3.6/site-packages/de_core_news_sm
    -->
    /Users/sob/Desktop/cs287/homeworks/env/lib/python3.6/site-packages/spacy/data/de

    You can now load the model via spacy.load('de')



In [ ]:
# load in tokenizers for German and English
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [10]:
# create utility functions to tokenize them
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [11]:
# add a beginning of sentence token and end of sentence token for target language only
BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

In [12]:
# for purposes of this hw, only gather sentences of max length 20
MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
print(train.fields)
print(len(train))
print(vars(train[0]))

downloading de-en.tgz
.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2010.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2010.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2011.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2011.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2012.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2012.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2013.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2013.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2014.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2014.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.dev2012.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TEDX.dev2012.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2013.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2013.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2014.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2014.de-en.en.xml
.data/iwslt/de-en/train.tags.de-en.de
.data/iwslt/de-e

In [13]:
# replace tokens that occur less than 5 times with unk tokens, take rest as our vocabulary
MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) #vocab index for <s>, </s>

[('.', 113253), (',', 67237), ('ist', 24189), ('die', 23778), ('das', 17102), ('der', 15727), ('und', 15622), ('Sie', 15085), ('es', 13197), ('ich', 12946)]
Size of German vocab 13353
[('.', 113433), (',', 59512), ('the', 46029), ('to', 29177), ('a', 27548), ('of', 26794), ('I', 24887), ('is', 21775), ("'s", 20630), ('that', 19814)]
Size of English vocab 11560
2 3


In [14]:
# make our batch and val iterators
BATCH_SIZE = 32
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

In [15]:
# example of what a batch looks like
batch = next(iter(train_iter))
print("Source")
print(batch.src)
print("Target")
print(batch.trg)

Source
Variable containing:

Columns 0 to 10 
    99   4715   1104    150     39     77      0     26     87      9      0
    22    635    123   8758   4142    272    707      4     21    278      0
  5912      5      0   1279    657     10     24     18     74    156    249
    80   6403      4      0     24      3      0     41      3      0    239
   731    907      0     62     32   3119      0      0     27      0   1488
  5383     18      3   3510     25     15    127     37     90     29     15
     8      3     31      0      0     89      6      5    117     56      0
    11    927   5934    294   5141      8      0    527   6207    286      3
  8760    198   7186      3   2247      7    149     48      8   3235    167
   174    178    101   2601      8   8966      0    308      0     68    144
   537    170   7248     79     15   1307     14   5863     34      0   3532
    34     34    972     84      0    858    507   3153      7      8     18
   731   4353      0     57   

In [26]:
""" our goal is to output predict the 100 most probable 3-gram that will begin 
    the target sentence. The submission format will be as follows, where each 
    word in the 3-gram will be separated by "|", and each 3-gram will be separated by space."""
!head ../data/source_test.txt

Als ich in meinen 20ern war , hatte ich meine erste Psychotherapie-Patientin .
Ich war Doktorandin und studierte Klinische Psychologie in Berkeley .
Sie war eine 26-jÃ¤hrige Frau namens Alex .
Und als ich das hÃ¶rte , war ich erleichtert .
Meine Kommilitonin bekam nÃ¤mlich einen Brandstifter als ersten Patienten .
Und ich bekam eine Frau in den 20ern , die Ã¼ber Jungs reden wollte .
Das kriege ich hin , dachte ich mir .
Aber ich habe es nicht hingekriegt .
Arbeit kam spÃ¤ter , Heiraten kam spÃ¤ter , Kinder kamen spÃ¤ter , selbst der Tod kam spÃ¤ter .
Leute in den 20ern wie Alex und ich hatten nichts als Zeit .
